In [3]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    LeafSystem,
    MeshcatVisualizerCpp,
    MultilayerPerceptron,
    Parser,
    PerceptronActivationType,
    RandomGenerator,
    RigidTransform,
    RotationMatrix,
    SceneGraph,
    Simulator,
    StartMeshcat,
    ZeroOrderHold,
)
from pydrake.examples.pendulum import PendulumGeometry, PendulumPlant
from underactuated import FindResource, running_as_notebook
from underactuated.optimizers import Adam
from value_iteration import *

In [2]:
mescat = StartMeshcat()

Meshcat is now available at http://localhost:7001
